In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import itertools
from tqdm import tqdm_notebook
from scipy.stats import spearmanr
from common import *

In [2]:
train = pd.read_csv(f'data/train.csv')
y = train[TARGETS].values
ix = np.where(train.groupby("question_body")["host"].transform("count") == 1)[0]

y_albert = pd.read_csv('oofs/siamese_albert_1_comb_oofs.csv', index_col=0).values
y_bert = pd.read_csv('oofs/siamese_bert_6_oofs.csv', index_col=0).values
y_roberta = pd.read_csv('oofs/siamese_roberta_1_comb_oofs.csv', index_col=0).values
# y_xlnet = pd.read_csv('oofs/siamese_xlnet_1_comb_oofs.csv', index_col=0).values
y_xlnet = pd.read_csv('oofs/siamese_xlnet_2_comb_oofs.csv', index_col=0).values

In [3]:
def evaluate(y, y_pred, verbose=False, exclude=[]):
    score = 0
    for i in range(y.shape[1]):
        if i not in exclude:
            col_score = spearmanr(y[:, i], y_pred[:, i])[0]
            if verbose:
                print(TARGETS[i], np.round(col_score, 3))
            score += col_score/(y.shape[1]-len(exclude))
    return np.round(score, 4)


ds = [2, 4, 8, 16, 32, 64, None]
ws = [0, 1, 2, 4]


def scale(x, d):
    if d:
        return (x//(1/d))/d
    return x

dws4 = list(itertools.product(ds, ds, ws, ws, ws, ws))


def transform(y_albert, y_roberta, y_bert, y_xlnet, params, c):
    d_global, d_local, w_albert, w_roberta, w_bert, w_xlnet = params
    y_temp = scale(y_albert[:, c], d_local)*w_albert +\
                 scale(y_roberta[:, c], d_local)*w_roberta +\
                 scale(y_bert[:, c], d_local)*w_bert +\
                 scale(y_xlnet[:, c], d_local)*w_xlnet
    y_temp /= w_albert + w_roberta + w_bert + w_xlnet
    y_temp = scale(y_temp, d_global)
    return y_temp


def f(c):
    max_score = spearmanr(y[:, c], y_roberta[:, c])[0]
    best_index = -1
    for i, params in enumerate(dws4):
        y_temp = transform(y_albert, y_roberta, y_bert, y_xlnet, params, c)
        score = spearmanr(y[:, c], y_temp)[0]
        if score > max_score:
            max_score = score
            best_index = i
            
    return best_index

In [4]:
import multiprocessing

pool = multiprocessing.Pool(15)
out = pool.map(f, range(30))

In [5]:
y_combined = np.zeros(y.shape)

for c in range(30):
    print(c, TARGETS[c], dws4[out[c]])
    y_combined[:, c] = transform(y_albert, y_roberta, y_bert, y_xlnet, dws4[out[c]], c)

0 question_asker_intent_understanding (64, 32, 2, 1, 2, 2)
1 question_body_critical (None, 64, 4, 1, 4, 1)
2 question_conversational (4, None, 2, 2, 1, 2)
3 question_expect_short_answer (16, None, 1, 2, 2, 4)
4 question_fact_seeking (16, 64, 1, 4, 4, 4)
5 question_has_commonly_accepted_answer (4, None, 4, 4, 2, 1)
6 question_interestingness_others (32, None, 1, 0, 1, 2)
7 question_interestingness_self (16, 64, 2, 1, 4, 4)
8 question_multi_intent (8, 32, 2, 1, 2, 2)
9 question_not_really_a_question (32, 32, 4, 1, 4, 1)
10 question_opinion_seeking (16, None, 1, 4, 1, 4)
11 question_type_choice (32, 4, 4, 4, 1, 2)
12 question_type_compare (4, None, 0, 2, 0, 1)
13 question_type_consequence (32, 8, 0, 1, 0, 4)
14 question_type_definition (8, 8, 0, 4, 1, 0)
15 question_type_entity (4, 64, 1, 1, 1, 1)
16 question_type_instructions (16, None, 2, 2, 1, 4)
17 question_type_procedure (32, 16, 1, 4, 0, 4)
18 question_type_reason_explanation (16, None, 4, 4, 2, 1)
19 question_type_spelling (16, Non

In [6]:
evaluate(y, y_combined, True)

question_asker_intent_understanding 0.401
question_body_critical 0.681
question_conversational 0.511
question_expect_short_answer 0.323
question_fact_seeking 0.381
question_has_commonly_accepted_answer 0.482
question_interestingness_others 0.369
question_interestingness_self 0.523
question_multi_intent 0.61
question_not_really_a_question 0.166
question_opinion_seeking 0.497
question_type_choice 0.773
question_type_compare 0.569
question_type_consequence 0.323
question_type_definition 0.669
question_type_entity 0.614
question_type_instructions 0.792
question_type_procedure 0.384
question_type_reason_explanation 0.686
question_type_spelling 0.584
question_well_written 0.54
answer_helpful 0.288
answer_level_of_information 0.477
answer_plausible 0.185
answer_relevance 0.198
answer_satisfaction 0.387
answer_type_instructions 0.773
answer_type_procedure 0.328
answer_type_reason_explanation 0.7
answer_well_written 0.252


0.4822

In [7]:
evaluate(y, y_combined, True, exclude=[9,19])

question_asker_intent_understanding 0.401
question_body_critical 0.681
question_conversational 0.511
question_expect_short_answer 0.323
question_fact_seeking 0.381
question_has_commonly_accepted_answer 0.482
question_interestingness_others 0.369
question_interestingness_self 0.523
question_multi_intent 0.61
question_opinion_seeking 0.497
question_type_choice 0.773
question_type_compare 0.569
question_type_consequence 0.323
question_type_definition 0.669
question_type_entity 0.614
question_type_instructions 0.792
question_type_procedure 0.384
question_type_reason_explanation 0.686
question_well_written 0.54
answer_helpful 0.288
answer_level_of_information 0.477
answer_plausible 0.185
answer_relevance 0.198
answer_satisfaction 0.387
answer_type_instructions 0.773
answer_type_procedure 0.328
answer_type_reason_explanation 0.7
answer_well_written 0.252


0.4899

In [8]:
[dws4[out[c]] for c in range(30)]

[(64, 32, 2, 1, 2, 2),
 (None, 64, 4, 1, 4, 1),
 (4, None, 2, 2, 1, 2),
 (16, None, 1, 2, 2, 4),
 (16, 64, 1, 4, 4, 4),
 (4, None, 4, 4, 2, 1),
 (32, None, 1, 0, 1, 2),
 (16, 64, 2, 1, 4, 4),
 (8, 32, 2, 1, 2, 2),
 (32, 32, 4, 1, 4, 1),
 (16, None, 1, 4, 1, 4),
 (32, 4, 4, 4, 1, 2),
 (4, None, 0, 2, 0, 1),
 (32, 8, 0, 1, 0, 4),
 (8, 8, 0, 4, 1, 0),
 (4, 64, 1, 1, 1, 1),
 (16, None, 2, 2, 1, 4),
 (32, 16, 1, 4, 0, 4),
 (16, None, 4, 4, 2, 1),
 (16, None, 1, 0, 2, 0),
 (None, 64, 1, 1, 4, 4),
 (32, 8, 2, 2, 0, 1),
 (32, 32, 2, 4, 1, 1),
 (64, 64, 1, 1, 0, 0),
 (8, 64, 2, 1, 2, 4),
 (None, None, 2, 2, 0, 1),
 (16, 64, 2, 2, 2, 1),
 (32, None, 1, 4, 1, 2),
 (32, 64, 2, 2, 0, 1),
 (32, None, 2, 1, 4, 4)]